In [1]:
import numpy as np
np.set_printoptions(linewidth = 128, precision = 3, formatter = {"bool" : lambda b : '#' if b else '_'})

In [2]:
# Numpy supporting point class
class Point:
    # Needs a coordinate
    def __init__(self, x : int, y : int):
        self.x = x
        self.y = y
        self.dict = {'y': y, 'x': x}
        self.npar = np.array([y, x], int)
        self.tupl = (x, y)
    # String representation
    def __str__(self) -> str:
        return str(self.tupl)
    # Generic representation (just uses __str__)
    def __repr__(self) -> str:
        return self.__str__()

In [3]:
# Numpy supporting rectangle class
class Rectangle:
    # Needs a coordinate for the northwest corner (nearest to origin), a width, and height
    def __init__(self, x : int, y : int, w : int, h : int):
        self.origin = Point(x, y)
        self.height = abs(h)
        self.width = abs(w)
        self.area = self.height * self.width
        self.refreshEdgeCells()
    # String representation
    def __str__(self) -> str:
        return "A {: 4d} by {: 4d} Rectangle cornered at ({: 4d}, {: 4d}).".format(
            self.width, self.height, self.origin.x, self.origin.y
        )
    # Generic representation (just uses __str__)
    def __repr__(self) -> str:
        return self.__str__()
    # Make six sets detailing the coordinates of cells in the rectangle on the edge of it
    def refreshEdgeCells(self):
        yi = self.origin.y
        xi = self.origin.x
        self.corners = {
            Point(xi + xo, yi + yo)
            for xo in (0, self.width - 1)
            for yo in (0, self.height - 1)
        }
        self.edgeCellsNorth = {
            Point(xe, yi)
            for xe in range(xi + 1, xi + self.width - 1)
        }
        self.edgeCellsWest = {
            Point(xi, ye)
            for ye in range(yi + 1, yi + self.height - 1)
        }
        self.edgeCellsSouth = {
            Point(xe, yi + self.height - 1)
            for xe in range(xi + 1, xi + self.width - 1)
        }
        self.edgeCellsEast = {
            Point(xi + self.width - 1, ye)
            for ye in range(yi + 1, yi + self.height - 1)
        }
        
        self.edgeCells = self.corners | self.edgeCellsNorth | self.edgeCellsWest \
            | self.edgeCellsSouth | self.edgeCellsEast
    # Get a binary numpy mask array with the rectangle filled in, to arbitrary frame size
    def getMaskFill(self, fw : int = 0, fh : int = 0) -> np.array:
        if fw == 0 or fh == 0:
            fw = self.origin.x + self.width
            fh = self.origin.y + self.height
        
        M = np.zeros((fh, fw), bool)
        
        M[
            self.origin.y : self.origin.y + self.height,
            self.origin.x : self.origin.x + self.width
        ] = 1
        
        return M
    # Get a binary numpy mask array with the rectangle's edge only, to arbitrary frame size
    def getMaskEdge(self, fw : int = 0, fh : int = 0) -> np.array:
        M = self.getMaskFill(fw, fh)
        
        M[
            self.origin.y + 1 : self.origin.y + self.height - 1,
            self.origin.x + 1 : self.origin.x + self.width - 1
        ] = 0
        
        return M
    # Use the above mask method to determine if this rectangle overlaps with another
    def overlaps(self, other) -> bool:
        w = max(self.origin.x + self.width, other.origin.x + other.width)
        h = max(self.origin.y + self.height, other.origin.y + other.height)
        return np.count_nonzero(self.getMaskFill(w, h) & other.getMaskFill(w, h)) > 0
    # Turn the overlap method into an operator (not commutative!)
    def __and__(self, other) -> bool:
        return self.overlaps(other)
    # Compute the percentage of how much of this rectangle overlaps with another
    def percentOverlap(self, other) -> float:
        w = max(self.origin.x + self.width, other.origin.x + other.width)
        h = max(self.origin.y + self.height, other.origin.y + other.height)
        return np.count_nonzero(self.getMaskFill(w, h) & other.getMaskFill(w, h)) / self.area
    # Determine the center cell of the rectangle
    def getCentroid(self) -> np.array:
        return self.origin.npar + np.array([self.height // 2, self.width // 2], int)
    # Determine the bearing towards another rectangle
    def getAzimuth(self, other) -> float:
        vector = self.getCentroid() - other.getCentroid()
        #print(vector, np.arctan2(vector[1], vector[0]) * 180. / np.pi, np.arctan2(vector[1], vector[0]) * 180. / np.pi - 90.)
        """
        To go from trig angles (start at 3 o'clock, counterclockwise) to azimuths
        (start at 6 o'clock (not noon since y increseas downward in graphics),
        also counterclockwise), we need to rotate 90 degrees to the right
        to bring the trig 0-axis to meet the place for the azimuth 0-axis.
        Mathematcially, this is $\theta + 90$. Since we don't want negative azimuths,
        we mod by 360 to wind negatives around.
        """
        return ((np.arctan2(vector[0], vector[1]) * 180. / np.pi) + 90.) % 360.
    # Determine the closest wall that faces another rectangle
    def getNearestWall(self, other) -> set:
        a = self.getAzimuth(other)
        if a > 315. or a <= 45.:
            return self.edgeCellsSouth
        elif a > 45. and a <= 135.:
            return self.edgeCellsWest
        elif a > 135. and a <= 225.:
            return self.edgeCellsNorth
        elif a > 225. and a <= 315.:
            return self.edgeCellsEast
        return {}

In [4]:
?Rectangle

Init signature: Rectangle(x: int, y: int, w: int, h: int)
Docstring:      <no docstring>
Type:           type
Subclasses:     


In [5]:
help(Rectangle)

Help on class Rectangle in module __main__:

class Rectangle(builtins.object)
 |  Rectangle(x: int, y: int, w: int, h: int)
 |  
 |  Methods defined here:
 |  
 |  __and__(self, other) -> bool
 |      # Turn the overlap method into an operator (not commutative!)
 |  
 |  __init__(self, x: int, y: int, w: int, h: int)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self) -> str
 |      Return repr(self).
 |  
 |  __str__(self) -> str
 |      Return str(self).
 |  
 |  getAzimuth(self, other) -> float
 |      # Determine the bearing towards another rectangle
 |  
 |  getCentroid(self) -> <built-in function array>
 |      # Determine the center cell of the rectangle
 |  
 |  getMaskEdge(self, fw: int = 0, fh: int = 0) -> <built-in function array>
 |      # Get a binary numpy mask array with the rectangle's edge only, to arbitrary frame size
 |  
 |  getMaskFill(self, fw: int = 0, fh: int = 0) -> <built-in function array>
 |      # Get a binary nump

In [6]:
r = Rectangle(2, 4, 8, 6)

In [7]:
r

A    8 by    6 Rectangle cornered at (   2,    4).

In [8]:
r.area

48

In [9]:
r.origin

(2, 4)

In [10]:
r.corners

{(2, 4), (2, 9), (9, 4), (9, 9)}

In [11]:
r.edgeCellsNorth

{(3, 4), (4, 4), (5, 4), (6, 4), (7, 4), (8, 4)}

In [12]:
r.edgeCellsWest

{(2, 5), (2, 6), (2, 7), (2, 8)}

In [13]:
r.edgeCellsSouth

{(3, 9), (4, 9), (5, 9), (6, 9), (7, 9), (8, 9)}

In [14]:
r.edgeCellsEast

{(9, 5), (9, 6), (9, 7), (9, 8)}

In [15]:
r.edgeCells

{(2, 4),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 8),
 (2, 9),
 (3, 4),
 (3, 9),
 (4, 4),
 (4, 9),
 (5, 4),
 (5, 9),
 (6, 4),
 (6, 9),
 (7, 4),
 (7, 9),
 (8, 4),
 (8, 9),
 (9, 4),
 (9, 5),
 (9, 6),
 (9, 7),
 (9, 8),
 (9, 9)}

In [16]:
r.getMaskEdge()

array([[_, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _],
       [_, _, #, #, #, #, #, #, #, #],
       [_, _, #, _, _, _, _, _, _, #],
       [_, _, #, _, _, _, _, _, _, #],
       [_, _, #, _, _, _, _, _, _, #],
       [_, _, #, _, _, _, _, _, _, #],
       [_, _, #, #, #, #, #, #, #, #]])

In [17]:
r.getMaskFill()

array([[_, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _],
       [_, _, #, #, #, #, #, #, #, #],
       [_, _, #, #, #, #, #, #, #, #],
       [_, _, #, #, #, #, #, #, #, #],
       [_, _, #, #, #, #, #, #, #, #],
       [_, _, #, #, #, #, #, #, #, #],
       [_, _, #, #, #, #, #, #, #, #]])

In [18]:
r2 = Rectangle(5, 1, 4, 6)

In [19]:
r2.getMaskFill()

array([[_, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, #, #, #, #],
       [_, _, _, _, _, #, #, #, #],
       [_, _, _, _, _, #, #, #, #],
       [_, _, _, _, _, #, #, #, #],
       [_, _, _, _, _, #, #, #, #],
       [_, _, _, _, _, #, #, #, #]])

In [20]:
r.overlaps(r2)

True

In [21]:
r & r2

True

In [22]:
r3 = Rectangle(5, 1, 4, 3)

In [23]:
r3.getMaskFill()

array([[_, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, #, #, #, #],
       [_, _, _, _, _, #, #, #, #],
       [_, _, _, _, _, #, #, #, #]])

In [24]:
r & r3

False

In [25]:
r2 & r3

True

In [26]:
r2.area

24

In [27]:
r3.area

12

In [28]:
r.percentOverlap(r2)

0.25

In [29]:
r.percentOverlap(r3)

0.0

In [30]:
r2.percentOverlap(r)

0.5

In [31]:
r2.percentOverlap(r3)

0.5

In [32]:
r3.percentOverlap(r2)

1.0

In [33]:
r, r.getCentroid(), r2, r2.getCentroid(), r3, r3.getCentroid()

(A    8 by    6 Rectangle cornered at (   2,    4).,
 array([7, 6]),
 A    4 by    6 Rectangle cornered at (   5,    1).,
 array([4, 7]),
 A    4 by    3 Rectangle cornered at (   5,    1).,
 array([2, 7]))

In [34]:
r.getCentroid() - r2.getCentroid()

array([ 3, -1])

In [35]:
(np.arctan2(_[0], _[1]) * 180. / np.pi + 90.) % 360.

198.43494882292202

In [36]:
r.getAzimuth(r2)

198.43494882292202

In [37]:
np.array([(x, y) for x in range(-2, 3) for y in range(-2, 3)]).T

array([[-2, -2, -2, -2, -2, -1, -1, -1, -1, -1,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2],
       [-2, -1,  0,  1,  2, -2, -1,  0,  1,  2, -2, -1,  0,  1,  2, -2, -1,  0,  1,  2, -2, -1,  0,  1,  2]])

In [38]:
(np.arctan2(_[1], _[0]) * 180. / np.pi), ((np.arctan2(_[1], _[0]) * 180. / np.pi) - 90.) % 360., 

(array([-135.   , -153.435,  180.   ,  153.435,  135.   , -116.565, -135.   ,  180.   ,  135.   ,  116.565,  -90.   ,  -90.   ,
           0.   ,   90.   ,   90.   ,  -63.435,  -45.   ,    0.   ,   45.   ,   63.435,  -45.   ,  -26.565,    0.   ,   26.565,
          45.   ]),
 array([135.   , 116.565,  90.   ,  63.435,  45.   , 153.435, 135.   ,  90.   ,  45.   ,  26.565, 180.   , 180.   , 270.   ,
          0.   ,   0.   , 206.565, 225.   , 270.   , 315.   , 333.435, 225.   , 243.435, 270.   , 296.565, 315.   ]))

In [39]:
r2.getAzimuth(r)

18.43494882292201

In [40]:
r.getNearestWall(r2)

{(3, 4), (4, 4), (5, 4), (6, 4), (7, 4), (8, 4)}

In [41]:
r2.getNearestWall(r)

{(6, 6), (7, 6)}

In [42]:
r.getNearestWall(r3), r3.getNearestWall(r)

({(3, 4), (4, 4), (5, 4), (6, 4), (7, 4), (8, 4)}, {(6, 3), (7, 3)})

In [43]:
r3.getMaskEdge()

array([[_, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, #, #, #, #],
       [_, _, _, _, _, #, _, _, #],
       [_, _, _, _, _, #, #, #, #]])

In [44]:
r2.getMaskEdge()

array([[_, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, #, #, #, #],
       [_, _, _, _, _, #, _, _, #],
       [_, _, _, _, _, #, _, _, #],
       [_, _, _, _, _, #, _, _, #],
       [_, _, _, _, _, #, _, _, #],
       [_, _, _, _, _, #, #, #, #]])

In [45]:
r.getMaskEdge(10, 10) | r2.getMaskEdge(10, 10) | r3.getMaskEdge(10, 10)

array([[_, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, #, #, #, #, _],
       [_, _, _, _, _, #, _, _, #, _],
       [_, _, _, _, _, #, #, #, #, _],
       [_, _, #, #, #, #, #, #, #, #],
       [_, _, #, _, _, #, _, _, #, #],
       [_, _, #, _, _, #, #, #, #, #],
       [_, _, #, _, _, _, _, _, _, #],
       [_, _, #, _, _, _, _, _, _, #],
       [_, _, #, #, #, #, #, #, #, #]])

In [46]:
#?np.arctan2

In [47]:
help(Rectangle)

Help on class Rectangle in module __main__:

class Rectangle(builtins.object)
 |  Rectangle(x: int, y: int, w: int, h: int)
 |  
 |  Methods defined here:
 |  
 |  __and__(self, other) -> bool
 |      # Turn the overlap method into an operator (not commutative!)
 |  
 |  __init__(self, x: int, y: int, w: int, h: int)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self) -> str
 |      Return repr(self).
 |  
 |  __str__(self) -> str
 |      Return str(self).
 |  
 |  getAzimuth(self, other) -> float
 |      # Determine the bearing towards another rectangle
 |  
 |  getCentroid(self) -> <built-in function array>
 |      # Determine the center cell of the rectangle
 |  
 |  getMaskEdge(self, fw: int = 0, fh: int = 0) -> <built-in function array>
 |      # Get a binary numpy mask array with the rectangle's edge only, to arbitrary frame size
 |  
 |  getMaskFill(self, fw: int = 0, fh: int = 0) -> <built-in function array>
 |      # Get a binary nump

In [48]:
# Numpy supporting line class
class Line:
    # Needs a coordinate, a length, and an orientation
    def __init__(self, x : int, y : int, l : int, o : str):
        o = o.lower()[0]
        self.origin = Point(x, y)
        self.length = abs(l)
        self.orient = Point(
            -1 if o == 'w' else 1 if o == 'e' else 0,
            -1 if o == 'n' else 1 if o == 's' else 0
        )
    # Get a binary numpy mask array with the line drawn, arbitrary frame size
    def getMask(self, fw : int = 0, fh : int = 0) -> np.array:
        xi = self.origin.x # the abs-abs thing is for the slice to work properly
        xf = self.origin.x + self.length * self.orient.x + abs(abs(self.orient.x) - 1)
        yi = self.origin.y
        yf = self.origin.y + self.length * self.orient.y + abs(abs(self.orient.y) - 1)
        if fw == 0 or fh == 0:
            fw = max(xi, xf)
            fh = max(yi, yf)
        # Flip if need be for the slice to work properly
        if xi > xf:
            t = xf
            xf = xi + 1
            xi = t + 1
        if yi > yf:
            t = yf
            yf = yi + 1
            yi = t + 1
            
        M = np.zeros((fh, fw), bool)
        
        M[yi:yf, xi:xf] = 1
        
        return M
    # Determine the center cell of the line
    def getCentroid(self) -> np.array:
        return self.origin.npar + self.orient.npar * (self.length // 2)
    # Determine the cell at the end of the line
    def getEndpoint(self) -> np.array:
        return self.origin.npar + self.orient.npar * (self.length - 1)
    # Determine the bearing to another line (or rectangle, centroid mode only!)
    def getAzimuth(self, other, mode = 'c') -> float:
        if mode.lower()[0] == 'e':
            vector = self.getEndpoint() - other.getEndpoint()
        else:
            vector = self.getCentroid() - other.getCentroid()
        # See description in Rectangle class for why this works
        return ((np.arctan2(vector[0], vector[1]) * 180. / np.pi) + 90.) % 360.
    # Determine which cardinal direction leads closest to another line
    # (or rectangle, centroid mode only like above!)
    def getNearestOrientation(self, other, mode = 'c') -> tuple:
        a = self.getAzimuth(other, mode)
        if a > 315. or a <= 45.:
            return ('s', Point(0, 1))
        elif a > 45. and a <= 135.:
            return ('w', Point(-1, 0))
        elif a > 135. and a <= 225.:
            return ('n', Point(0, -1))
        elif a > 225. and a <= 315.:
            return ('e', Point(1, 0))
        return ()

In [49]:
l1 = Line(2, 2, 3, 'e')

In [50]:
l1.orient

(1, 0)

In [51]:
l1.getMask(20, 8)

array([[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, #, #, #, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _]])

In [52]:
l1.getCentroid()

array([2, 3])

In [53]:
l1.origin

(2, 2)

In [54]:
l1.getEndpoint()

array([2, 4])

In [55]:
l2 = Line(10, 4, 7, 'w')

In [56]:
l2.length, l2.length // 2

(7, 3)

In [57]:
l2.origin.npar, l2.getCentroid(), l2.getEndpoint()

(array([ 4, 10]), array([4, 7]), array([4, 4]))

In [58]:
l2.getMask(20, 8)

array([[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, #, #, #, #, #, #, #, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _]])

In [59]:
l1.getMask(20, 8) | l2.getMask(20, 8)

array([[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, #, #, #, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, #, #, #, #, #, #, #, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _]])

In [60]:
l2.getAzimuth(l1)

116.56505117707799

In [61]:
l2.getAzimuth(l1, mode = "endpoint")

180.0

In [62]:
l2.getNearestOrientation(l1)

('w', (-1, 0))

In [63]:
l2.getNearestOrientation(l1, 'e')

('n', (0, -1))

In [64]:
help(Point)

Help on class Point in module __main__:

class Point(builtins.object)
 |  Point(x: int, y: int)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, x: int, y: int)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self) -> str
 |      Return repr(self).
 |  
 |  __str__(self) -> str
 |      Return str(self).
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [65]:
help(Line)

Help on class Line in module __main__:

class Line(builtins.object)
 |  Line(x: int, y: int, l: int, o: str)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, x: int, y: int, l: int, o: str)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  getAzimuth(self, other, mode='c') -> float
 |      # Determine the bearing to another line (or rectangle, centroid mode only!)
 |  
 |  getCentroid(self) -> <built-in function array>
 |      # Determine the center cell of the line
 |  
 |  getEndpoint(self) -> <built-in function array>
 |      # Determine the cell at the end of the line
 |  
 |  getMask(self, fw: int = 0, fh: int = 0) -> <built-in function array>
 |      # Get a binary numpy mask array with the line drawn, arbitrary frame size
 |  
 |  getNearestOrientation(self, other, mode='c') -> tuple
 |      # Determine which cardinal direction leads closest to another line
 |      # (or rectangle, centroid mode only like above!)
 |  
 |  ---------------------